#EDA
# Credit Card Default Analysis

In [ ]:
# Import packages and modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# Read the Excel file
df = pd.read_excel("/content/default of credit card clients.xls", header=1)

# Print the shape of the DataFrame
print(df.shape)


# 2. Explore the data :


In [ ]:
# Examine the first 15 rows of the data
df.head(15)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.loc[0]

In [ ]:
#  there are 2 undocumented codes: 0,-2

pay = df[['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']]
pay_melt = pd.melt(pay)
print(pay_melt['value'].value_counts())

In [ ]:
df['default payment next month'].value_counts(normalize=True)

# 3. Data Cleaning:

In [ ]:
df.rename(columns={'PAY_0':'PAY_1','default payment next month':'DEFAULT_PAY'},inplace = True)
df.columns

In [ ]:
# Check if there is any duplicate IDs
condition = bool(df.duplicated(subset = 'ID').any())

if condition:
    print('There are duplicate IDs')
else:
    print('No duplicate IDs')

In [ ]:
# Use histogram to view the distribution of credit limit

plt.hist(df["LIMIT_BAL"])
plt.xlabel('Credit Limit ', fontweight='bold')
plt.ylabel(' Customers', fontweight='bold')
plt.show()

In [ ]:
# Visualize outliers in the distribution of credit limits
df["LIMIT_BAL"].plot(kind="box")
plt.xlabel('Credit Limit ', fontweight='bold')
plt.ylabel('Number of Customers', fontweight='bold')
plt.show()


In [ ]:
# Investigate the high credit limit outlier
# Based on other feature values, this record represents a customer with excellent payment behavior
# Hence, it is considered a legitimate observation and will not be removed

outliers = df.loc[df['LIMIT_BAL'] > 900000]
outliers


In [ ]:
df.describe()

In [ ]:
# Saving processed csv file for part 2 - modeling

df.to_csv("Credit Card Processed.csv", index=None)

In [ ]:
# check sex column
df["SEX"].value_counts()

In [ ]:
df["SEX"]=df["SEX"].map({1:'M',2:'F'}).astype('category')
df["SEX"].dtypes

In [ ]:
# eductaion column
df["EDUCATION"].unique()
df["EDUCATION"].value_counts()

In [ ]:
# Change values 4, 5, 6 to 0 and define 0 as 'others'
# 1=graduate school, 2=university, 3=high school, 0=others

df["EDUCATION"] = df["EDUCATION"].replace({4:0,5:0,6:0})
df["EDUCATION"].value_counts()

In [ ]:
# marriage column
df["MARRIAGE"].unique()

In [ ]:
df["MARRIAGE"].value_counts(normalize=True)

In [ ]:
df["MARRIAGE"] = df["MARRIAGE"].replace({0:3})
df["MARRIAGE"].value_counts(normalize=True)

# 4. Hypothesis and Findings

In [ ]:
df.set_index('ID', inplace = True)
df.head()

In [ ]:
def_condition = (df.PAY_1 >1) | (df.PAY_2 >1) | (df.PAY_3 >1) | (df.PAY_4 >1) | (df.PAY_5 >1) | (df.PAY_6 >1)
df.loc[def_condition, "HAS_DEF"] = 1
df.loc[df.HAS_DEF.isna(), "HAS_DEF"] = 0

In [ ]:
def create_stacked_bar(column_name):
    # Get the percentage of default by each group
    default_by_group = pd.crosstab(index=df['HAS_DEF'], columns=df[column_name], normalize='columns')

    # Round to 2 decimal places
    default_by_group = default_by_group.apply(lambda x: round(x, 2))

    labels = default_by_group.columns
    list1 = default_by_group.iloc[0].to_list()
    list2 = default_by_group.iloc[1].to_list()

    list1_name = "No default"
    list2_name = "Has default"
    title = f"Default by {column_name}"
    xlabel = column_name
    ylabel = "Default percentage"

    fig, ax = plt.subplots(figsize=(10, 5))
    bar_width = 0.5

    # Custom colors
    color1 = '#4CAF50'  # Green for No default
    color2 = '#F44336'  # Red for Has default

    ax1 = ax.bar(labels, list1, bar_width, label=list1_name, color=color1)
    ax2 = ax.bar(labels, list2, bar_width, bottom=list1, label=list2_name, color=color2)

    ax.set_title(title, fontweight="bold")
    ax.set_xlabel(xlabel, fontweight="bold")
    ax.set_ylabel(ylabel, fontweight="bold")
    ax.legend(loc="best")

    plt.xticks(list(range(len(labels))), labels, rotation=90)
    plt.yticks(fontsize=9)

    for r1, r2 in zip(ax1, ax2):
        h1 = r1.get_height()
        h2 = r2.get_height()
        plt.text(r1.get_x() + r1.get_width() / 2., h1 / 2., f"{h1:.0%}", ha="center", va="center", color="white", fontsize=9, fontweight="bold")
        plt.text(r2.get_x() + r2.get_width() / 2., h1 + h2 / 2., f"{h2:.0%}", ha="center", va="center", color="white", fontsize=9, fontweight="bold")

    plt.tight_layout()
    plt.show()


In [ ]:
create_stacked_bar('SEX')

In [ ]:
create_stacked_bar('EDUCATION')

In [ ]:
# Use boxplot to visualize credit limit grouped by education level

data = []
for i in [0,1,2,3]:
    temp = df.loc[df.EDUCATION == i, "LIMIT_BAL"]
    data.append(temp)

fig, ax = plt.subplots()
ax.boxplot(data)
ax.set_xticklabels(["others","grad school", "university", "high school"])

plt.show()

In [ ]:
create_stacked_bar('MARRIAGE')

In [ ]:
# Segment the 'AGE' column to 6 groups

bins= [21,30,40,50,60,70,80]
labels = ['20-30','30-40','40-50','50-60','60-70','70-80']
df['AGE'] = pd.cut(df['AGE'],bins=bins, labels=labels,right=False)

In [ ]:
# Plot a stacked bar chart to visualize default and non-default by age

create_stacked_bar('AGE')

In [ ]:
# Subset a dataframe with the records that have default

has_default = df[df['HAS_DEF']== 1]
default_trend = has_default[['PAY_6','PAY_5','PAY_4','PAY_3','PAY_2','PAY_1']].sum(axis=0)

# Draw a line chart to show the trend. The lower the number, the shorter delayed payment
fig,ax = plt.subplots()
ax.plot(default_trend)
plt.xticks(['PAY_6','PAY_5','PAY_4','PAY_3','PAY_2','PAY_1'],['Apr','May','Jun','Jul','Aug','Sep'])

plt.xlabel('Months in 2005',fontweight='bold')
plt.ylabel('Total delayed months',fontweight='bold')
plt.title('Delayed payment trend',fontweight='bold')

plt.show()

In [ ]:
# Make a boxplot to visualize credit limit and default payment next month
# 1: default next month; 0: no default next month

def0 = df.loc[df['DEFAULT_PAY'] == 0,'LIMIT_BAL']
def1 = df.loc[df['DEFAULT_PAY'] == 1,'LIMIT_BAL']

fig, ax = plt.subplots()
ax.boxplot([def0, def1],  showfliers=False)

ax.set_xticklabels(['No_default',"Default"],fontweight ='bold')
ax.set_ylabel('Credit limit',fontweight ='bold')
ax.set_title('Credit limit & default next month',fontweight ='bold')

plt.show()

In [ ]:
# Get statistic summary of bill statement columns
# The min numbers are negative

bill = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']]
bill.describe()

In [ ]:
# How many bills have negative amount?
# There are 3932 bills with negative amounts, each month has 599-688 records

bill_melt = pd.melt(bill, var_name = 'bill_cycle',value_name = 'amount')
neg_bill = bill_melt[bill_melt['amount']<0]
neg_bill.groupby('bill_cycle').count()

In [ ]:
# Get the average amount of negative bill each month
# Use USD/NTD = 30 to get the amount in USD. The average negative amount is $38-$102

neg_bill_mean_ndt = neg_bill.groupby('bill_cycle')['amount'].mean()

print('Average negative amounts in NTD are: \n')
print(neg_bill_mean_ndt)
print('\nAverage negative amounts in USD are: \n')
print(neg_bill_mean_ndt/30)

In [ ]:
# Is there any bill amount that is greater than credit limit?

condition1 = df['BILL_AMT1'] > df['LIMIT_BAL']
condition2 = df['BILL_AMT2'] > df['LIMIT_BAL']
condition3 = df['BILL_AMT3'] > df['LIMIT_BAL']
condition4 = df['BILL_AMT4'] > df['LIMIT_BAL']
condition5 = df['BILL_AMT5'] > df['LIMIT_BAL']
condition6 = df['BILL_AMT6'] > df['LIMIT_BAL']

large_bill = df[condition1 | condition2 |condition3 | condition4 | condition5 | condition6]
large_bill.shape

In [ ]:
large_bill["HAS_DEF"].value_counts()

In [ ]:
# Is there any customer who never had any transaction, meaning bill amount in 6 months remained as 0
# There are 870 customers whose bill amount was 0 in 6 months

bill_amt = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']]
no_transaction = bill_amt[bill_amt.sum(axis=1) ==0]
no_transaction

In [ ]:
# Check if those 870 customers defaulted in the next month (Oct.2005)
# The assumption is since they did not have any consumption, they shouldn't default, so there shouldn't be any '1'

no_transaction_de = df.loc[no_transaction.index,['DEFAULT_PAY']]
no_transaction_de

In [ ]:
# How many inactive customers had default payment next month
# 317 customers had default payment next month which is against common sense

no_transaction_de['DEFAULT_PAY'].value_counts()

#5. Statistical Analysis

Should this section be embeded to previous EDA step?

* Correlation between credit limit and bill amount: Pearson R
* Correlation between credit limit and payment default: t-test
* Correlation between education and payment default: chi-squared test
* Correlation between age and payment default: chi-squared test
* Correlation between sex and payment default: permutation test
* Correlation between sex and credit limit: t-test

In [ ]:
df['BILL_SUM'] = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1)
credit_bill = df[['LIMIT_BAL','BILL_SUM']]
credit_bill.corr()

In [ ]:
from scipy import stats

credit_limit = df['LIMIT_BAL']
bill_amount = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].sum(axis=1)

stats.pearsonr(credit_limit, bill_amount)

#### Correlation between credit limit and payment default.
Ideally, we should have customers' income data, but since this data is not available, we can assume higher income customers have higher credit limits. Therefore, we will verify if there is a correlation between credit limit and default payment using t-test.

- Null hypothesis: credit limit does not affect default likelihood.
- Alternative hypothesis: credit limit impact default likelihood.
- Set significance level $\alpha$ to 0.05

In [ ]:
no_def_limit = df.loc[df['HAS_DEF']==0,'LIMIT_BAL']
def_limit = df.loc[df['HAS_DEF']==1,'LIMIT_BAL']

In [ ]:
from scipy import stats
from scipy.stats import ttest_ind_from_stats

ttest_ind_from_stats(mean1=np.mean(no_def_limit),std1=np.std(no_def_limit),nobs1=len(no_def_limit),
                     mean2=np.mean(def_limit),std2=np.std(def_limit),nobs2=len(def_limit))

#### Correlation between education and payment default.

Previous visualization indicates education impacts default likelihood. We will use a Chi-squared test to verify this finding.
- Null hypothesis: education does not affect default likelihood.
- Alternative hypothesis: education impacts default likelihood.
- Set significance level $\alpha$ to 0.05

In [ ]:
# Get the contigence table of education and default data

edu_def_contigence = pd.crosstab(df['HAS_DEF'], df['EDUCATION'], margins=False)

In [ ]:
# Use Chi-squared test to test if education affects default likelihood.

from scipy.stats import chisquare
from scipy.stats import chi2_contingency
stat, p, dof, expected = chi2_contingency(edu_def_contigence)

p

#### Correlation between age and payment default.

Previous visualization indicates age impacts default likelihood. We will use a chi-squared test to verify this finding.
- Null hypothesis: age does not affect default likelihood.
- Alternative hypothesis: age impacts default likelihood.
- Set significance level $\alpha$ to 0.05

In [ ]:

age_def_contigence = pd.crosstab(df['HAS_DEF'], df['AGE'], margins=False)

In [ ]:
# Use chi-squared test to test if age affects default likelihood.

from scipy.stats import chisquare
from scipy.stats import chi2_contingency
stat, p, dof, expected = chi2_contingency(age_def_contigence)
p

#### Correlation between sex and payment default.

In previous data visualization, it appears males tend to default more than females. Does sex have any correlations with default or was this observation due to chance event? Let's find out with a permutation test on each group's default proportions and mean respectively.
- Null hypothesis: sex has no impact on default probability.
- Alternative hypothesis: sex has impact on default probability.
- Set significance level $\alpha$ to 0.05.

In [ ]:
# Get the true difference of 'HAS_DEF' proportions of two sex groups

male = df.loc[df['SEX']=='M','HAS_DEF']
female = df.loc[df['SEX']=='F','HAS_DEF']
true_diff_prop = male.sum()/len(male) - female.sum()/len(female)
true_diff_prop

In [ ]:
# Define functions to do permutation test

def permutation_sample(data1, data2):
    data = np.concatenate((data1,data2))
    permuted_data = np.random.permutation(data)
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def diff_of_prop(data1,data2):
    diff = data1.sum()/len(data1) - data2.sum()/len(data2)

    return diff

def draw_perm_reps(data1, data2, func, size=1):
    perm_replicates = np.empty(size)

    for i in range(size):
        perm_sample_1, perm_sample_2 = permutation_sample(data1,data2)
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

In [ ]:
# Draw 10000 permutation tests

perm_replicates = draw_perm_reps(male,female,diff_of_prop,size=10000)

In [ ]:
# Calculate p value - the probability of getting a mean difference equal or greater than the observed mean difference

p = np.sum(perm_replicates >= true_diff_prop) / len(perm_replicates)
p

#### Correlation between credit limit and sex.

Since we know the credit limit is strongly correlated with default probability. Let's see if sex plays any role in credit limit and default likelihood.
- Null hypothesis: sex has no impact on credit limit.
- Alternative hypothesis: sex has impact on credit limit.
- Set significance level $\alpha$ to 0.05.

In [ ]:
# Calculate the true mean difference of credit limit

male_limit= df.loc[df['SEX']=='M','LIMIT_BAL']
female_limit = df.loc[df['SEX']=='F','LIMIT_BAL']
print(np.mean(male_limit),np.mean(female_limit))

In [ ]:
# Use t-test to verify if different sex groups have the same mean of credit limit.

from scipy import stats
from scipy.stats import ttest_ind_from_stats

ttest_ind_from_stats(mean1=np.mean(male_limit),std1=np.std(male_limit),nobs1=len(male_limit),
                     mean2=np.mean(female_limit),std2=np.std(female_limit),nobs2=len(female_limit))